In [1]:
import numpy as np
import pandas as pd

не забудьте перенести в папку с этим .ipynb-файлом 'train_data.parquet', полученный в "01 Preparation". Именно с помощью него будем формировать сбалансированную обучающую выборку.

In [2]:
data = pd.read_parquet('train_data.parquet')

In [3]:
data['category_id'].value_counts()

11937    6590
14922    3709
13651    1463
13143    1460
12980    1222
         ... 
12808       2
12901       1
11549       1
11875       1
12836       1
Name: category_id, Length: 874, dtype: int64

In [4]:
data['category_id'].value_counts().describe()

count     874.000000
mean      104.256293
std       300.107191
min         1.000000
25%        14.000000
50%        38.000000
75%        99.000000
max      6590.000000
Name: category_id, dtype: float64

Выберем по 50 объектов каждого класса

C OverSample посложнее

In [5]:
#Prepare for OverSampling
def prepare_over_sample(df, n_sample):
    desc_cats = df['category_id'].value_counts().to_frame()
    desc_cats = desc_cats[desc_cats['category_id']<=n_sample]
    
    ids_for_oversample = desc_cats.index
    
    pre_over_sample_idxs = []
    for idx in ids_for_oversample:
        indices_current_cat = df[df['category_id']==idx].index
        
        pre_over_sample_idxs.append(list(indices_current_cat))
    
    return_ix = np.array([a for b in pre_over_sample_idxs for a in b])
    return df.loc[return_ix]

In [6]:
data_for_over = prepare_over_sample(data, 50)

In [7]:
data_for_over['category_id'].value_counts()

12773    50
15068    50
11636    50
13847    50
14682    49
         ..
12808     2
12901     1
11549     1
11875     1
12836     1
Name: category_id, Length: 505, dtype: int64

In [8]:
data_for_over['category_id'].value_counts().describe()

count    505.000000
mean      19.877228
std       13.201310
min        1.000000
25%        9.000000
50%       17.000000
75%       29.000000
max       50.000000
Name: category_id, dtype: float64

In [27]:
#Сохраним промежуточный результат
#data_for_over.to_parquet('train_before_oversample.parquet')

In [61]:
data_for_over.head()

category_id                                               Docs
1689         12773  осветители и вспышки селфи кольцо на телефон с...
4345         12773  осветители и вспышки лампа кольцо для селфи ми...
5580         12773  осветители и вспышки селфи кольцо лампа для те...
8892         12773  осветители и вспышки кольцевая лампа  светодио...
11479        12773  осветители и вспышки селфи лампа для телефона ...

In [67]:
def bootstrap_oversample(df, n_samples):
    cats = np.unique(df['category_id'].values)
    
    samples = []
    
    for cat in cats:
        current_group = df[df['category_id']==cat]
        for i in range(n_samples):
            sample = current_group.sample()
            samples.append(list(sample.values[0]))
            
    df_new = pd.DataFrame(samples)
    
    
    return df_new.rename(columns = {0:'category_id', 1:'Docs'})

In [69]:
df_oversampled = bootstrap_oversample(data_for_over, 50)

In [71]:
df_oversampled['category_id'].value_counts().describe()

count    505.0
mean      50.0
std        0.0
min       50.0
25%       50.0
50%       50.0
75%       50.0
max       50.0
Name: category_id, dtype: float64

In [75]:
df_oversampled.head()

category_id                                               Docs
0         2600  пляжная одежда пляжное платье рубашка промокод...
1         2600  пляжная одежда женская пляжная накидка яркая н...
2         2600  пляжная одежда рубашка пляжное платье женская ...
3         2600  пляжная одежда пляжное платье цвет   синий100 ...
4         2600  пляжная одежда женская пляжная накидка яркая н...

In [76]:
#Сохраняем oversampled_data
df_oversampled.to_parquet('train_oversampled_data.parquet')